In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Psoriasis"
cohort = "GSE158448"

# Input paths
in_trait_dir = "../../input/GEO/Psoriasis"
in_cohort_dir = "../../input/GEO/Psoriasis/GSE158448"

# Output paths
out_data_file = "../../output/preprocess/Psoriasis/GSE158448.csv"
out_gene_data_file = "../../output/preprocess/Psoriasis/gene_data/GSE158448.csv"
out_clinical_data_file = "../../output/preprocess/Psoriasis/clinical_data/GSE158448.csv"
json_path = "../../output/preprocess/Psoriasis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Multiple IL-17 family cytokines signaling through IL-17RA drive inflammatory pathways in psoriasis"
!Series_summary	"The IL-23/IL-17 immune axis is of central importance in psoriasis. However, the contribution of IL-17 family cytokines other than IL-17A to drive skin inflammation in psoriasis has not been fully established. To further elucidate the role of individual IL-17 family cytokines in psoriasis, we investigated their expression and localization in psoriasis skin at the mRNA and protein level. Moreover, we investigated the gene expression signatures induced by individual IL-17 family cytokines in human skin ex vivo as well as modulation of responses induced by the combination of IL-17 family cytokines in human keratinocytes by brodalumab, a human monoclonal antibody targeting the IL-17RA, versus the IL-17A blocking antibody ixekizumab.     We demonstrate that IL-17A, IL-17AF, IL-17F and IL-17C are expressed at increased levels in psoriasis 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# Analyze the data availability and define conversion functions

# 1. Gene Expression Data Availability
# From the background information, this appears to be a gene expression study related to IL-17 cytokines in psoriasis
# The study design mentions "60 human skin samples" which likely contains gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# 2.1 Data Availability for trait, age, and gender
# Trait data: We can infer this from the 'treatment' field (row 4)
trait_row = 4  # The treatment field indicates whether samples are from patients with specific treatments

# Age data: Not available in the sample characteristics
age_row = None

# Gender data: Not available in the sample characteristics 
gender_row = None

# 2.2 Data Type Conversion Functions

def convert_trait(value):
    """
    Convert treatment values to binary indicating whether it's untreated (0) or treated (1)
    """
    if not isinstance(value, str):
        return None
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if 'untreated' in value.lower():
        return 0  # Untreated sample
    elif 'IL17' in value:
        return 1  # Treated with some form of IL17
    else:
        return None  # Unknown

# Since age and gender data are not available, we don't need conversion functions for them
convert_age = None
convert_gender = None

# 3. Save Metadata - Initial Filtering
# Check if both gene and trait data are available
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Create a DataFrame from the sample characteristics dictionary provided in the task
    # The sample characteristics dictionary was already provided in the task output
    sample_characteristics = {
        0: ['sample_id: 01', 'sample_id: 02', 'sample_id: 03', 'sample_id: 04', 'sample_id: 05', 'sample_id: 06', 'sample_id: 07', 'sample_id: 08', 'sample_id: 09', 'sample_id: 10', 'sample_id: 11', 'sample_id: 12', 'sample_id: 13', 'sample_id: 14', 'sample_id: 15', 'sample_id: 16', 'sample_id: 17', 'sample_id: 18', 'sample_id: 19', 'sample_id: 20', 'sample_id: 21', 'sample_id: 22', 'sample_id: 23', 'sample_id: 24', 'sample_id: 25', 'sample_id: 26', 'sample_id: 27', 'sample_id: 28', 'sample_id: 29', 'sample_id: 30'], 
        1: ['donor_id: 2', 'donor_id: 3', 'donor_id: 4', 'donor_id: 6', 'donor_id: 7', 'donor_id: 8', 'donor_id: 9', 'donor_id: 11', 'donor_id: 12', 'donor_id: 14', 'donor_id: 16', 'donor_id: 17', 'donor_id: 19', 'donor_id: 20', 'donor_id: 21', 'donor_id: 22', 'donor_id: 23', 'donor_id: 24', 'donor_id: 25', 'donor_id: 28', 'donor_id: 30', 'donor_id: 31', 'donor_id: 32', 'donor_id: 33', 'donor_id: 35', 'donor_id: 37', 'donor_id: 38', 'donor_id: 39', 'donor_id: 41', 'donor_id: 42'], 
        2: ['rna_integrity: 9.1', 'rna_integrity: 9.4', 'rna_integrity: 9', 'rna_integrity: 9.2', 'rna_integrity: 8.4', 'rna_integrity: 8.9', 'rna_integrity: 9.3', 'rna_integrity: 8.6', 'rna_integrity: 8.8', 'rna_integrity: 9.5', 'rna_integrity: 8', 'rna_integrity: 8.7', 'rna_integrity: 9.6', 'rna_integrity: 9.7', 'rna_integrity: 9.8', 'rna_integrity: 8.3', 'rna_integrity: 7.3'], 
        3: ['batch: batch_1', 'batch: batch_2', 'batch: batch_3', 'batch: batch_4'], 
        4: ['treatment: untreated', 'treatment: IL17A_1_ng_ml', 'treatment: IL17A_10_ng_ml', 'treatment: IL17A_100_ng_ml', 'treatment: IL17AF_10_ng_ml', 'treatment: IL17AF_100_ng_ml', 'treatment: IL17F_10_ng_ml', 'treatment: IL17F_100_ng_ml', 'treatment: IL17C_10_ng_ml', 'treatment: IL17C_100_ng_ml', 'treatment: IL17E_10_ng_ml', 'treatment: IL17E_100_ng_ml']
    }
    
    # Convert to DataFrame
    clinical_data = pd.DataFrame.from_dict(sample_characteristics, orient='index')
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the data
    print("Preview of the selected clinical features:")
    print(preview_df(selected_clinical_df))
    
    # Save clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of the selected clinical features:
{0: [0.0], 1: [1.0], 2: [1.0], 3: [1.0], 4: [1.0], 5: [1.0], 6: [1.0], 7: [1.0], 8: [1.0], 9: [1.0], 10: [1.0], 11: [1.0], 12: [nan], 13: [nan], 14: [nan], 15: [nan], 16: [nan], 17: [nan], 18: [nan], 19: [nan], 20: [nan], 21: [nan], 22: [nan], 23: [nan], 24: [nan], 25: [nan], 26: [nan], 27: [nan], 28: [nan], 29: [nan]}
Clinical data saved to ../../output/preprocess/Psoriasis/clinical_data/GSE158448.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Re-identify the SOFT and matrix files to ensure we have the correct paths
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract the gene expression data from the matrix file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers)
print("\nFirst 20 gene/probe identifiers:")
print(gene_data.index[:20])

# 4. Print the dimensions of the gene expression data
print(f"\nGene data dimensions: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")

# Note: we keep is_gene_available as True since we successfully extracted gene expression data
is_gene_available = True



First 20 gene/probe identifiers:
Index(['16650001', '16650003', '16650005', '16650007', '16650009', '16650011',
       '16650013', '16650015', '16650017', '16650019', '16650021', '16650023',
       '16650025', '16650027', '16650029', '16650031', '16650033', '16650035',
       '16650037', '16650041'],
      dtype='object', name='ID')

Gene data dimensions: 53617 genes × 60 samples


### Step 4: Gene Identifier Review

In [5]:
# These identifiers appear to be numeric probe IDs, not human gene symbols
# They are likely platform-specific identifiers that need to be mapped to gene symbols

# Looking at the format (purely numeric IDs starting with "1665"), these are 
# not standard human gene symbols which typically have alphabetic characters

# They are most likely probe IDs from a microarray platform that need to be 
# converted to standard gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Inspect the SOFT file structure to understand the annotation format
# Read the first few lines of the SOFT file to examine its structure
import gzip
print("Preview of SOFT file content:")
with gzip.open(soft_file, 'rt') as f:
    for i, line in enumerate(f):
        print(line.strip())
        if i >= 20:  # Print first 20 lines to understand structure
            break
print("...\n")

# 3. Try different approaches to extract gene annotation data
# First, let's try the default method to see what's actually in the file
gene_annotation = get_gene_annotation(soft_file)
print("Gene annotation dataframe using default method:")
print(f"Shape: {gene_annotation.shape}")
print(f"Columns: {gene_annotation.columns.tolist()}")
print(gene_annotation.head(3))

# 4. Check if there's another section in the file that might contain the mapping
# Look for platform annotation information in the SOFT file
print("\nSearching for platform annotation section in SOFT file...")
with gzip.open(soft_file, 'rt') as f:
    platform_lines = []
    capture = False
    for i, line in enumerate(f):
        if "^PLATFORM" in line:
            capture = True
            platform_lines.append(line.strip())
        elif capture and line.startswith("!platform_table_begin"):
            platform_lines.append(line.strip())
            for j in range(10):  # Capture the next 10 lines to understand the table structure
                try:
                    platform_line = next(f).strip()
                    platform_lines.append(platform_line)
                except StopIteration:
                    break
            break
    
    print("\n".join(platform_lines))

# Maintain gene availability status as True based on previous steps
is_gene_available = True


Preview of SOFT file content:
^DATABASE = GeoMiame
!Database_name = Gene Expression Omnibus (GEO)
!Database_institute = NCBI NLM NIH
!Database_web_link = http://www.ncbi.nlm.nih.gov/geo
!Database_email = geo@ncbi.nlm.nih.gov
^SERIES = GSE158448
!Series_title = Multiple IL-17 family cytokines signaling through IL-17RA drive inflammatory pathways in psoriasis
!Series_geo_accession = GSE158448
!Series_status = Public on Apr 07 2021
!Series_submission_date = Sep 23 2020
!Series_last_update_date = Apr 10 2021
!Series_pubmed_id = 33792895
!Series_summary = The IL-23/IL-17 immune axis is of central importance in psoriasis. However, the contribution of IL-17 family cytokines other than IL-17A to drive skin inflammation in psoriasis has not been fully established. To further elucidate the role of individual IL-17 family cytokines in psoriasis, we investigated their expression and localization in psoriasis skin at the mRNA and protein level. Moreover, we investigated the gene expression signatur

Gene annotation dataframe using default method:
Shape: (3270697, 20)
Columns: ['ID', 'probeset_id', 'seqname', 'strand', 'start', 'stop', 'total_probes', 'gene_assignment', 'mrna_assignment', 'swissprot', 'unigene', 'GO_biological_process', 'GO_cellular_component', 'GO_molecular_function', 'pathway', 'protein_domains', 'crosshyb_type', 'category', 'GB_ACC', 'SPOT_ID']
         ID probeset_id seqname strand  start   stop  total_probes  \
0  16657436    16657436    chr1      +  12190  13639          25.0   
1  16657440    16657440    chr1      +  29554  31109          28.0   
2  16657445    16657445    chr1      +  69091  70008           8.0   

                                     gene_assignment  \
0  NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-As...   
1  ENST00000473358 // MIR1302-11 // microRNA 1302...   
2  NM_001005484 // OR4F5 // olfactory receptor, f...   

                                     mrna_assignment  \
0  NR_046018 // RefSeq // Homo sapiens DEAD/H (As...   
1  ENST0000

### Step 6: Gene Identifier Mapping

In [7]:
# 1. From the gene annotation preview, we need to determine which columns to use
# Looking at the gene annotation data, we can see:
# - The 'ID' column contains the probe identifiers (e.g., 16657436) which match our gene expression data
# - The 'gene_assignment' column appears to contain gene symbols and other gene information

# 2. Extract the mapping information from the annotation dataframe
# We need the ID column (probe identifiers) and gene_assignment column (which contains gene names)
mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='gene_assignment')

# Print a sample of the mapping dataframe to verify
print("\nSample of the gene mapping dataframe:")
print(mapping_df.head())
print(f"Shape of mapping dataframe: {mapping_df.shape}")

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(expression_df=gene_data, mapping_df=mapping_df)

# Print information about the resulting gene expression data
print("\nAfter mapping to gene symbols:")
print(f"Gene expression data dimensions: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")
print("\nSample of gene expression data (first 5 genes, first 3 samples):")
print(gene_data.iloc[:5, :3])

# Maintain gene availability status
is_gene_available = True if gene_data.shape[0] > 0 else False



Sample of the gene mapping dataframe:
         ID                                               Gene
0  16657436  NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-As...
1  16657440  ENST00000473358 // MIR1302-11 // microRNA 1302...
2  16657445  NM_001005484 // OR4F5 // olfactory receptor, f...
3  16657447                                                ---
4  16657450  AK302511 // LOC100132062 // uncharacterized LO...
Shape of mapping dataframe: (53617, 2)



After mapping to gene symbols:
Gene expression data dimensions: 81076 genes × 60 samples

Sample of gene expression data (first 5 genes, first 3 samples):
      GSM4800737  GSM4800738  GSM4800739
Gene                                    
A-     12.128679   12.520490   12.287928
A-2     1.807653    1.736914    1.684175
A-52    3.715036    3.666027    3.689625
A-E     1.212786    0.902884    1.320547
A-I     3.026220    2.757676    2.957934


### Step 7: Data Normalization and Linking

In [8]:
# 1. Check if gene data is available after mapping
if gene_data.shape[0] == 0:
    print("Error: Gene expression matrix is empty after mapping.")
    # Mark the dataset as not usable due to lack of gene expression data
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=False,  # No usable gene data
        is_trait_available=True,
        is_biased=True,
        df=pd.DataFrame(),
        note="Failed to map probe IDs to gene symbols. The annotation format may not be compatible with the extraction methods."
    )
    print("Dataset deemed not usable due to lack of gene expression data.")
else:
    # Only proceed with normalization if we have gene data
    print("Normalizing gene symbols...")
    gene_data_normalized = normalize_gene_symbols_in_index(gene_data)
    print(f"Gene data shape after normalization: {gene_data_normalized.shape[0]} genes × {gene_data_normalized.shape[1]} samples")

    # Save the normalized gene data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    gene_data_normalized.to_csv(out_gene_data_file)
    print(f"Normalized gene expression data saved to {out_gene_data_file}")
    
    # Extract clinical features from the original data source
    print("Extracting clinical features from the original source...")
    # Get background information and clinical data again
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    print("Extracted clinical features preview:")
    print(preview_df(selected_clinical_df))
    print(f"Clinical data shape: {selected_clinical_df.shape}")
    
    # Save the extracted clinical features
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical features saved to {out_clinical_data_file}")
    
    # Link clinical and genetic data
    print("Linking clinical and genetic data...")
    linked_data = geo_link_clinical_genetic_data(selected_clinical_df, gene_data_normalized)
    print(f"Linked data shape: {linked_data.shape}")
    
    # Check if the linked data has adequate data
    if linked_data.shape[0] == 0 or linked_data.shape[1] <= 4:  # 4 is an arbitrary small number
        print("Error: Linked data has insufficient samples or features.")
        is_usable = validate_and_save_cohort_info(
            is_final=True,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=True,
            is_trait_available=True,
            is_biased=True,
            df=linked_data,
            note="Failed to properly link gene expression data with clinical features."
        )
        print("Dataset deemed not usable due to linking failure.")
    else:
        # Handle missing values systematically
        print("Handling missing values...")
        linked_data_clean = handle_missing_values(linked_data, trait_col=trait)
        print(f"Data shape after handling missing values: {linked_data_clean.shape}")
        
        # Check if there are still samples after missing value handling
        if linked_data_clean.shape[0] == 0:
            print("Error: No samples remain after handling missing values.")
            is_usable = validate_and_save_cohort_info(
                is_final=True,
                cohort=cohort,
                info_path=json_path,
                is_gene_available=True,
                is_trait_available=True,
                is_biased=True,
                df=pd.DataFrame(),
                note="All samples were removed during missing value handling."
            )
            print("Dataset deemed not usable as all samples were filtered out.")
        else:
            # Check if the dataset is biased
            print("\nChecking for bias in feature variables:")
            is_biased, linked_data_final = judge_and_remove_biased_features(linked_data_clean, trait)
            
            # Conduct final quality validation
            is_usable = validate_and_save_cohort_info(
                is_final=True,
                cohort=cohort,
                info_path=json_path,
                is_gene_available=True,
                is_trait_available=True,
                is_biased=is_biased,
                df=linked_data_final,
                note="Dataset contains gene expression data for Crohn's Disease patients, examining response to Infliximab treatment."
            )
            
            # Save linked data if usable
            if is_usable:
                os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
                linked_data_final.to_csv(out_data_file)
                print(f"Linked data saved to {out_data_file}")
                print(f"Final dataset shape: {linked_data_final.shape}")
            else:
                print("Dataset deemed not usable for trait association studies, linked data not saved.")

Normalizing gene symbols...
Gene data shape after normalization: 23274 genes × 60 samples


Normalized gene expression data saved to ../../output/preprocess/Psoriasis/gene_data/GSE158448.csv
Extracting clinical features from the original source...
Extracted clinical features preview:
{'GSM4800737': [0.0], 'GSM4800738': [0.0], 'GSM4800739': [0.0], 'GSM4800740': [0.0], 'GSM4800741': [0.0], 'GSM4800742': [1.0], 'GSM4800743': [1.0], 'GSM4800744': [1.0], 'GSM4800745': [1.0], 'GSM4800746': [1.0], 'GSM4800747': [1.0], 'GSM4800748': [1.0], 'GSM4800749': [1.0], 'GSM4800750': [1.0], 'GSM4800751': [1.0], 'GSM4800752': [1.0], 'GSM4800753': [1.0], 'GSM4800754': [1.0], 'GSM4800755': [1.0], 'GSM4800756': [1.0], 'GSM4800757': [1.0], 'GSM4800758': [1.0], 'GSM4800759': [1.0], 'GSM4800760': [1.0], 'GSM4800761': [1.0], 'GSM4800762': [1.0], 'GSM4800763': [1.0], 'GSM4800764': [1.0], 'GSM4800765': [1.0], 'GSM4800766': [1.0], 'GSM4800767': [1.0], 'GSM4800768': [1.0], 'GSM4800769': [1.0], 'GSM4800770': [1.0], 'GSM4800771': [1.0], 'GSM4800772': [1.0], 'GSM4800773': [1.0], 'GSM4800774': [1.0], 'GSM4800

Clinical features saved to ../../output/preprocess/Psoriasis/clinical_data/GSE158448.csv
Linking clinical and genetic data...
Linked data shape: (60, 23275)
Handling missing values...


Data shape after handling missing values: (60, 23275)

Checking for bias in feature variables:
For the feature 'Psoriasis', the least common label is '0.0' with 5 occurrences. This represents 8.33% of the dataset.
The distribution of the feature 'Psoriasis' in this dataset is fine.

A new JSON file was created at: ../../output/preprocess/Psoriasis/cohort_info.json


Linked data saved to ../../output/preprocess/Psoriasis/GSE158448.csv
Final dataset shape: (60, 23275)
